In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install resampy

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 11.5 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import librosa

import numpy as np
from scipy.io import wavfile as wav
from sklearn.decomposition import PCA

def traverse_subfolders(subfolders):
    audio_files = []
    subfolder_names = []

    for subfolder in subfolders:
        subfolder_name = os.path.basename(subfolder)
        audio_files.extend(get_audio_files(subfolder))
        subfolder_names.extend([subfolder_name] * len(get_audio_files(subfolder)))

    return audio_files, subfolder_names

def get_audio_files(subfolder):
    audio_files = []

    for root, dirs, files in os.walk(subfolder):
        for file in files:
            if file.endswith(".ogg"):
                audio_files.append(os.path.join(root, file))

    return audio_files

def features_extractor(file_name):
    audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
    required_length = 7 * sample_rate #each sample takes 7 seconds.
    if len(audio) < required_length:
        # Calculate the number of repeats needed to exceed or meet the required length
        repeats = int(np.ceil(required_length / len(audio)))
        audio = np.tile(audio, repeats)[:required_length]  # Repeat and trim to exact length
    elif len(audio) > required_length:
        audio = audio[:required_length]  # Trim to the required length if longer
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

    return mfccs_scaled_features

def process_audio_files(audio_files, subfolder_names, n_components):
    data = []

    for audio_file, subfolder_name in zip(audio_files, subfolder_names):
        compressed_data = features_extractor(audio_file)  # Extract MFCC features

        data.append([audio_file, compressed_data, subfolder_name])

    df = pd.DataFrame(data, columns=["File", "Compressed Data", "Subfolder Name"])
    return df

def compress_audio_pca(audio_data, n_components):
    # Apply PCA
    pca = PCA(n_components=n_components)
    compressed_data = pca.fit_transform(audio_data)

    return compressed_data


import plotly.express as px

def visualize_pca_interactive(compressed_data, subfolder_names):
    df_vis = pd.DataFrame({
        'PC1': compressed_data[:, 0],
        'PC2': compressed_data[:, 1],
        'Subfolder Name': subfolder_names
    })

    fig = px.scatter(df_vis, x='PC1', y='PC2', color='Subfolder Name',
                     title='Interactive PCA Visualization',
                     labels={'PC1': 'Principal Component 1', 'PC2': 'Principal Component 2'},
                     hover_data={'PC1': False, 'PC2': False, 'Subfolder Name': True})
    fig.show()

origin = '/content/drive/MyDrive/Dombyra dataset/Sounds/'
subfolders_orig = [os.path.join(origin, folder) for folder in os.listdir(origin) if os.path.isdir(os.path.join(origin, folder))]


n_components = 2

audio_files, subfolder_names = traverse_subfolders(subfolders_orig)

#print(len(audio_files))
#print(subfolder_names)
df = process_audio_files(audio_files, subfolder_names, n_components)

In [ ]:
audio_data = np.array(df['Compressed Data'].tolist())
subfolder_names = df['Subfolder Name'].values

# Visualize PCA of compressed data interactively
n_components = 2

# Apply PCA with the adjusted number of components
compressed_data_pca = compress_audio_pca(audio_data, n_components)
visualize_pca_interactive(compressed_data_pca, subfolder_names)

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import LabelEncoder

def compress_audio_lda(audio_data, labels, n_components):
    lda = LDA(n_components=n_components)
    compressed_data = lda.fit_transform(audio_data, labels)
    return compressed_data

# Assuming you have already processed your audio files and obtained a DataFrame df
audio_data = np.array(df['Compressed Data'].tolist())
subfolder_names = df['Subfolder Name'].values

# Convert class labels to numerical values
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(subfolder_names)

# Set the number of components for LDA
n_components = 2

# Apply LDA with the specified number of components
compressed_data_lda = compress_audio_lda(audio_data, labels, n_components)

# Visualize LDA of compressed data interactively
visualize_pca_interactive(compressed_data_lda, subfolder_names)